In [20]:
%pip -q install google-genai
# Instalar Framework de agentes do Google
%pip install -q google-adk


import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente 1: Buscador de Destinos --- #
##########################################
def agente_buscador(destino, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de viagens da agência Lima's Tur, uma das maiores agências de viagem do Brasil.
        A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar informações relevantes apenas sobre a cidade de destino informada abaixo. Você deve ignorar a cidade de origem.
        Informações tais como pontos turísticos, restaurantes, atrações, acomodações
        o que fazer na cidade de destino informada, melhor época do ano para visitar, etc.
        Foque em informações relevantes, com base na quantidade e entusiasmo das avaliações sobre ela.
        Se um ponto turístico, restaurante ou acomodação tiver poucas notícias ou avaliações entusiasmadas, é possível que não seja tão relevante assim
        e pode ser substituído por outros que tenham mais.
        Essas informações relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        É imprescindível a inclusão dos links para os websites dos pontos turísticos, restaurantes, atrações, acomodações, etc. caso estejam disponíveis.
        """,
        description="Agente que busca informações no Google sobre determinado destino",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Destino: {destino}\nData de hoje: {data_de_hoje}"

    destino_buscado = call_agent(buscador, entrada_do_agente_buscador)
    return destino_buscado

################################################
# --- Agente 2: Planejador da viagem --- #
################################################
def agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de viagens da agência Lima's Tur, uma das maiores agências de viagem do Brasil, especialista em
        encontrar passagens aéreas e terrestres mais baratas, acomodações, restaurantes, passeios, etc. para um destino específico.
        Com base na cidade de destino informada, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano de viagem o mais completo possível,
        com valores em reais (R$) de passagens aéres e terrestres, caso tenham. Você também pode usar o Google (google_search) para encontrar mais
        informações sobre cidade de destino e aprofundar. Esse plano de viagem deverá ser completo e detalhado e deverá levar em consideração a cidade de origem
        e o estado de origem, para traçar a rota e informar os valores de transporte. Informe os valores sempre em reais do Brasil (R$).
        Ao final, você irá escolher as melhores opções com base nas suas pesquisas
        e retornar esse plano de viagem completo, com seus pontos mais relevantes.
        Ao informar as opções, prestar muita atenção na separação das palavras, para que fique mais fácil a leitura.
        """,
        description="Agente que planeja viagens",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Cidade destino:{cidade_destino}\nEstado destino: {estado_destino}\nCidade de Origem: {cidade_origem}\nEstado Origem: {estado_origem}"
    # Executa o agente
    plano_de_viagem = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_viagem

######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar e-mails marketing da agência Lima's Tur.
            Você escreve e-mails marketing para uma empresa de viagens especializada em destinos nacionais.
            Utilize o destino fornecido no plano de destino e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de um e-mail cativante sobre o tema indicado.
            O e-mail deve ser engajador, informativo, e deve despertar no leitor uma vontade de conhecer a cidade,
            com linguagem simples e incluir uma chamada para ação no final.
            """,
        description="Agente redator de e-mails marketing engajadores"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho_gerado = call_agent(redator, entrada_do_agente_redator)
    return rascunho_gerado

##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em e-mails para marketing direto.
            Você tem um público diversificado, desde 18 anos até 70 anos. Use um tom de escrita adequado, envolvente, cativante,
            nem formal demais e nem coloquial demais.
            Revise o rascunho do e-mail abaixo sobre o destino indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado


data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Planejamento de Viagens e criação de E-mail Marketing com 4 Agentes 🚀")




########## Lógica do sistma de agentes ##########

while True:
    cidade_destino = input("❓ Por favor, informe a cidade de destino: ").strip()
    if cidade_destino:
        break
    else:
        print("Você esqueceu de digitar a cidade de destino!")

while True:
    estado_destino = input("Informe o estado de destino: ").strip()
    if estado_destino:
        break
    else:
        print("Você esqueceu de digitar o estado de destino!")

while True:
    cidade_origem = input("Informe a cidade de origem: ").strip()
    if cidade_origem:
        break
    else:
        print("Você esqueceu de digitar a cidade de origem!")

while True:
    estado_origem = input("Informe o estado de origem: ").strip()
    if estado_origem:
        break
    else:
        print("Você esqueceu de digitar o estado de origem!")

# --- Obter o destino do Usuário ---


destino = f"{cidade_destino}, {estado_destino} - {cidade_origem}, {estado_origem}"

if cidade_destino and estado_destino and cidade_origem and estado_origem:
    print(f"Maravilha! Vamos então criar o seu roteiro de viagem para {cidade_destino.upper()} - {estado_destino.upper()}")

    destino_buscado = agente_buscador(destino, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(destino_buscado))
    print("--------------------------------------------------------------")

    plano_de_viagem = agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_viagem))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(destino, plano_de_viagem)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(destino, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Planejamento de Viagens e criação de E-mail Marketing com 4 Agentes 🚀
❓ Por favor, informe a cidade de destino: resende
Informe o estado de destino: rj
Informe a cidade de origem: rio de janeiro
Informe o estado de origem: rj
Maravilha! Vamos então criar o seu roteiro de viagem para RESENDE - RJ

--- 📝 Resultado do Agente 1 (Buscador) ---



> Olá! A Lima's Tur tem ótimas opções para sua viagem a Resende, RJ. Confira as dicas que separei para você aproveitar ao máximo a cidade:
> 
> **O que fazer em Resende:**
> 
> *   **Pontos Turísticos:**
>     *   **Academia Militar das Agulhas Negras (AMAN):** É um dos maiores complexos militares do mundo e a maior da América Latina. A AMAN forma os oficiais combatentes do Exército Brasileiro. Dentro da Academia, você pode visitar o **Teatro da AMAN**, um moderno espaço cultural com capacidade para 2.821 pessoas.
>     *   **Parque das Águas:** Localizado no coração da cidade, o parque oferece uma área de lazer com pista para caminhada, academia ao ar livre, área para patinação e slackline, rampa de skate, quadra de areia e um lago com peixes ornamentais.
>     *   **Ponte Nilo Peçanha (Ponte Velha):** Construída em 1905, a ponte é feita de estrutura metálica importada da Bélgica.
>     *   **Centro Histórico de Resende:** Explore os casarões, praças, pontes e igrejas do século XIX. A **Fazenda do Castelo** é um dos prédios históricos mais bem conservados.
>     *   **Parque Nacional do Itatiaia:** Ideal para quem gosta de aventuras ao ar livre.
> *   **Eventos:**
>     *   **Festival da Cereja 2025:** A partir de 31 de maio de 2025.
>     *   **Exposição de “Pintura Expressionista e Cubista”:** A partir de 30 de maio de 2025.
>     *   **Fim de Semana Gastronómico:** A partir de 23 de maio de 2025.
>     *   **Festa do Pinhão:** Tradicional evento que acontece em Visconde de Mauá, com música, dança, gastronomia e a Cavalgada. Aconteceu entre 9 e 11 de maio de 2025.
>     *   **Carnaval 2025:**Shows na Avenida Rita Ferreira da Rocha, desfile de blocos no Centro Histórico e programação infantil no Ecoparque. Aconteceu entre 1 e 4 de março de 2025.
> *   **Dicas:**
>     *   A melhor época para visitar Resende é quando você puder mergulhar nas descobertas fascinantes que vão desde picos nevados até águas cristalinas.
> 
> **Onde Comer:**
> 
> Resende oferece uma variedade de restaurantes com diferentes tipos de cozinha.
> 
> *   **Restaurantes:**
>     *   Celeiro Bar e Restaurante:
>     *   Choperia Coronel Park
>     *   General Chopperia
> 
> **Onde se hospedar:**
> 
> *   **Sítio Vale dos Reis:** Com piscina ao ar livre e cozinha compartilhada.
> *   **Refúgio na Serrinha do Alambari - Penedo -RJ:** Acomodação com Wi-Fi grátis, piscina ao ar livre e jardim.
> *   **Chalé Floresta Mauá:** Oferece vista para a montanha, Wi-Fi grátis e estacionamento privativo grátis.
> *   **Casa da Pedra Serrinha:** Com piscina ao ar livre, jardim e bar.
> 
> Espero que estas sugestões ajudem você a planejar uma viagem inesquecível para Resende!


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Olá! Com base nas informações fornecidas, preparei um plano de viagem detalhado para você de Rio de Janeiro (RJ) para Resende (RJ). Vamos explorar as opções de transporte, acomodação, restaurantes e passeios para que você tenha uma experiência incrível!
> 
> 
> ## Plano de Viagem Detalhado: Rio de Janeiro (RJ) para Resende (RJ)
> 
> ### Transporte
> 
> *   **Ônibus:** A forma mais comum e acessível de ir do Rio de Janeiro para Resende é de ônibus.
>     *   **Preço médio da passagem:** R$ 70,51 a R$ 90, dependendo da empresa e do tipo de ônibus.
>     *   **Tempo de viagem:** Aproximadamente 2 horas e 40 minutos a 3 horas e 25 minutos.
>     *   **Empresas:** Viação Cidade do Aço, Viação Sampaio.
>     *   **Horários:** Várias opções diárias a partir da Rodoviária Novo Rio.
>     *   **Dica:** Compre a passagem com antecedência para garantir melhores preços e horários.
> *   **Carona:** Blablacar oferece caronas a partir de R$ 30,00.
> 
> ### Acomodação
> 
> Resende oferece diversas opções de hotéis e pousadas para todos os gostos e orçamentos. Algumas sugestões incluem:
> 
> *   **River Park Hotel:** Bem localizado, próximo ao centro da cidade.
> *   **Resende Inn:** Boa avaliação e custo-benefício.
> *   **Cegil Hotel Boulevard:** Hotel bem cuidado com academia e bom café da manhã.
> *   **Espigão Palace Hotel:** Próximo à rodoviária, com WiFi e café da manhã gratuitos.
> *   **Scala Residence Hotel:** Com piscina ao ar livre e WiFi gratuito.
> *   **Pé da Serra Hotel:** Localizado em um local mais afastado.
> *   **Hotel Vilar Formoso:** Oferece ar condicionado, Wi-Fi e piscina ao ar livre.
> *   **Pousada do Sol:** Possui sauna.
> 
> ### Restaurantes
> 
> Resende conta com uma variedade de restaurantes que oferecem desde a culinária local até opções internacionais.
> 
> *   **Loazô Restô:** Especializado em frutos do mar.
> *   **Restaurante Jardim Secreto - Penedo RJ:**.
> *   **Restaurante Peixe Boi:** Também especializado em frutos do mar.
> *   **Restaurante Celeiro - Resende:** Oferece um cardápio variado.
> *   **Restaurante Trem Azul:**.
> *   **Pizzai Delivery:** Para quem gosta de pizza.
> *   **Forneria Firenze:**.
> *   **Choperia General 750:**.
> *   **Uai Su:**.
> *   **Paladar da Montanha:**.
> *   **O Fino da Roça:**.
> *   **Restaurante Trem de Minas:**.
> *   **Padaria e Restaurante Pão Quente:**.
> *   **Marmitaria BH:**.
> *   **Rubaiyat:**.
> *   **Pronto Chef:**.
> 
> ### Pontos Turísticos
> 
> Resende e seus arredores oferecem diversas atrações para todos os gostos.
> 
> *   **Parque Nacional do Itatiaia:** Ideal para os amantes da natureza, com trilhas e cachoeiras.
> *   **Penedo:** Distrito com arquitetura e cultura finlandesa.
> *   **Visconde de Mauá:** Região montanhosa com pousadas charmosas e restaurantes.
> *   **Vila da Fumaça:** Famosa pela Cachoeira da Fumaça, a maior do Rio de Janeiro.
> *   **Academia Militar das Agulhas Negras (AMAN):** Possui um museu militar e um teatro.
> *   **Parque das Águas:** Opção de lazer com áreas verdes e atividades.
> *   **Museu Militar da AMAN:** Para quem se interessa por história militar.
> *   **Centro Histórico de Resende:** Com construções históricas e igrejas.
> *   **Igreja Matriz de Nossa Senhora da Conceição:** Importante patrimônio histórico.
> *   **Fundação Casa da Cultura Macedo Miranda:** Espaço cultural com exposições e eventos.
> *   **Ponte Nilo Peçanha (Ponte Velha):** Cartão postal da cidade.
> *   **Sítio Cachoeiras do Alcantilado:** Com nove cachoeiras e trilhas.
> *   **Cachoeira da Fumaça:** A maior queda d'água do Rio de Janeiro, com 200 metros.
> 
> ### Dicas Extras
> 
> *   **Clima:** Resende tem um clima tropical, com temperaturas mais amenas no inverno e mais altas no verão.
> *   **Gastronomia:** Experimente os pratos típicos da região, como a truta e a comida mineira.
> *   **Compras:** O Resende Shopping oferece diversas opções de lojas.
> *   **Ecoturismo:** Aproveite as diversas cachoeiras e trilhas da região.
> 
> Espero que este plano de viagem detalhado ajude você a aproveitar ao máximo sua visita a Resende! Se precisar de mais informações ou tiver alguma dúvida, é só perguntar.


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> **Assunto:** Descubra os encantos de Resende, RJ: Seu refúgio perfeito te espera! 🏞️
> 
> Olá, [Nome]!
> 
> Já pensou em escapar da agitação da cidade e se conectar com a natureza exuberante do Rio de Janeiro? 🤔 Resende, um tesouro escondido no estado, te espera com paisagens de tirar o fôlego e experiências inesquecíveis!
> 
> Imagine-se explorando o majestoso Parque Nacional do Itatiaia, com suas trilhas desafiadoras e cachoeiras revigorantes. 🤩 Ou, quem sabe, se aventurando na Vila da Fumaça, lar da impressionante Cachoeira da Fumaça, a maior do Rio de Janeiro, com seus 200 metros de queda d'água!
> 
> E não para por aí! Resende te convida a conhecer:
> 
> *   **Penedo:** um charmoso distrito com arquitetura e cultura finlandesa que encanta a todos.
> *   **Visconde de Mauá:** uma região montanhosa com pousadas aconchegantes e restaurantes deliciosos, perfeita para relaxar e desfrutar da gastronomia local.
> *   **Academia Militar das Agulhas Negras (AMAN):** um local histórico com um museu militar fascinante e um teatro imponente.
> 
> Além disso, Resende oferece uma variedade de restaurantes que vão te conquistar pelo paladar, desde a culinária local até opções internacionais. 😋 E para garantir o seu descanso, a cidade conta com hotéis e pousadas para todos os gostos e orçamentos.
> 
> 🚌 Chegar em Resende é fácil! A partir do Rio de Janeiro, você pode optar por um ônibus confortável, com passagens a partir de R$70,51, ou até mesmo encontrar uma carona no Blablacar por cerca de R$30,00.
> 
> Não perca a oportunidade de vivenciar momentos únicos em Resende! ✨
> 
> **Clique aqui** para saber mais sobre nossos pacotes e planejar sua viagem dos sonhos! ➡️ [link para a página de Resende no site da agência]
> 
> Estamos te esperando para tornar essa aventura realidade! 😉
> 
> Atenciosamente,
> 
> [Seu Nome]
> 
> Lima's Tur - Sua Agência de Viagens Especializada em Destinos Nacionais 🇧🇷


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
